In [17]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
import logging
import plotly.graph_objects as go
from datetime import timedelta

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Download VADER lexicon
nltk.download('vader_lexicon', quiet=True)

class SentimentAnalyzer:
    def __init__(self):
        self.sia = SentimentIntensityAnalyzer()

    def analyze_sentiment(self, text: str) -> float:
        if not text:
            return 0.0
        return self.sia.polarity_scores(text)['compound']

class TranscriptProvider:
    def __init__(self):
        self.transcripts = {
            'AAPL': "We had strong growth and positive outlook for the next quarter.",
            'GOOGL': "We faced challenges and missed our revenue targets.",
            'MSFT': "The company continues to innovate and expand market share.",
            'AMZN': "Costs have increased significantly, which will impact margins.",
            'FB': "We are optimistic about user engagement but ad revenues are down."
        }

    def get_transcript(self, ticker: str, date: pd.Timestamp) -> str:
        return self.transcripts.get(ticker, "No transcript available")

class EarningsCallSentimentStrategy:
    def __init__(self, sentiment_analyzer: SentimentAnalyzer, transcript_provider: TranscriptProvider):
        self.sentiment_analyzer = sentiment_analyzer
        self.transcript_provider = transcript_provider

    def execute(self, date: pd.Timestamp, universe: List[str]) -> Tuple[List[str], List[str], Dict[str, float]]:
        long_positions = []
        short_positions = []
        sentiment_scores = {}
        
        for ticker in universe:
            transcript = self.transcript_provider.get_transcript(ticker, date)
            sentiment = self.sentiment_analyzer.analyze_sentiment(transcript)
            sentiment_scores[ticker] = sentiment
            
            logging.info(f"Ticker: {ticker}, Sentiment: {sentiment:.2f}")
            
            if sentiment > 0:
                long_positions.append(ticker)
            elif sentiment < 0:
                short_positions.append(ticker)
    
        return long_positions, short_positions, sentiment_scores

class PriceDataProvider:
    def get_price_data(self, tickers: List[str], start_date: pd.Timestamp, end_date: pd.Timestamp) -> pd.DataFrame:
        date_range = pd.date_range(start=start_date, end=end_date, freq='D')
        price_data = {}
        for ticker in tickers:
            prices = 100 + np.cumsum(np.random.randn(len(date_range)) * 0.5)
            price_data[ticker] = prices
        return pd.DataFrame(price_data, index=date_range)

class PortfolioSimulator:
    def __init__(self, price_data_provider: PriceDataProvider):
        self.price_data_provider = price_data_provider

    def simulate_returns(self, long_positions: List[str], short_positions: List[str], 
                         start_date: pd.Timestamp, end_date: pd.Timestamp) -> pd.DataFrame:
        all_tickers = list(set(long_positions + short_positions))
        price_data = self.price_data_provider.get_price_data(all_tickers, start_date, end_date)
        
        returns = price_data.pct_change()
        
        portfolio_returns = pd.Series(0, index=price_data.index)
        
        for ticker in long_positions:
            portfolio_returns += returns[ticker]
        
        for ticker in short_positions:
            portfolio_returns -= returns[ticker]
        
        cumulative_returns = (1 + portfolio_returns).cumprod() - 1
        return pd.DataFrame({'Portfolio': cumulative_returns, **price_data})

def plot_interactive_returns(returns_data: pd.DataFrame, sentiment_scores: Dict[str, float]):
    fig = go.Figure()

    # Plot portfolio returns
    fig.add_trace(go.Scatter(x=returns_data.index, y=returns_data['Portfolio'],
                             mode='lines', name='Portfolio'))

    # Plot individual stock prices
    for column in returns_data.columns:
        if column != 'Portfolio':
            sentiment = sentiment_scores.get(column, 0)
            color = 'green' if sentiment > 0 else 'red' if sentiment < 0 else 'gray'
            fig.add_trace(go.Scatter(x=returns_data.index, y=returns_data[column],
                                     mode='lines', name=f"{column} (Sentiment: {sentiment:.2f})",
                                     line=dict(color=color), visible='legendonly'))

    fig.update_layout(title='Portfolio Returns and Stock Prices',
                      xaxis_title='Date',
                      yaxis_title='Returns / Price',
                      legend_title='Stocks')

    fig.show()

def main():
    universe = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'FB']
    start_date = pd.Timestamp('2023-10-01')
    end_date = start_date + timedelta(days=365)  # Simulate for 1 year

    sentiment_analyzer = SentimentAnalyzer()
    transcript_provider = TranscriptProvider()
    price_data_provider = PriceDataProvider()

    strategy = EarningsCallSentimentStrategy(sentiment_analyzer, transcript_provider)
    long_positions, short_positions, sentiment_scores = strategy.execute(start_date, universe)

    logging.info(f"Long positions: {long_positions}")
    logging.info(f"Short positions: {short_positions}")

    # Simulate and plot returns
    portfolio_simulator = PortfolioSimulator(price_data_provider)
    returns_data = portfolio_simulator.simulate_returns(long_positions, short_positions, start_date, end_date)
    plot_interactive_returns(returns_data, sentiment_scores)

if __name__ == "__main__":
    main()

2024-10-21 21:49:57,429 - INFO - Ticker: AAPL, Sentiment: 0.86
2024-10-21 21:49:57,430 - INFO - Ticker: GOOGL, Sentiment: -0.23
2024-10-21 21:49:57,430 - INFO - Ticker: MSFT, Sentiment: 0.77
2024-10-21 21:49:57,431 - INFO - Ticker: AMZN, Sentiment: 0.27
2024-10-21 21:49:57,431 - INFO - Ticker: FB, Sentiment: 0.39
2024-10-21 21:49:57,432 - INFO - Long positions: ['AAPL', 'MSFT', 'AMZN', 'FB']
2024-10-21 21:49:57,432 - INFO - Short positions: ['GOOGL']


In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Function to simulate sentiment data
def simulate_sentiment(tickers, days):
    return pd.DataFrame(np.random.randn(days, len(tickers)), columns=tickers)

# Function to calculate volatility
def calculate_volatility(returns, window=20):
    return returns.rolling(window=window).std() * np.sqrt(252)

# Function to implement the strategy
def sentiment_volatility_strategy(sentiment_data, price_data, volatility_data, n_components=3):
    # Standardize sentiment data
    scaler = StandardScaler()
    scaled_sentiment = scaler.fit_transform(sentiment_data)
    
    # Apply PCA
    pca = PCA(n_components=n_components)
    sentiment_factors = pca.fit_transform(scaled_sentiment)
    
    # Calculate overall sentiment score
    sentiment_score = pd.DataFrame(sentiment_factors).mean(axis=1)
    
    # Determine long and short positions
    long_positions = (sentiment_score > sentiment_score.quantile(0.7)) & (volatility_data < volatility_data.quantile(0.3))
    short_positions = (sentiment_score < sentiment_score.quantile(0.3)) & (volatility_data > volatility_data.quantile(0.7))
    
    # Calculate strategy returns
    position_size = 1 / len(price_data.columns)  # Equal weight for each position
    long_returns = price_data.pct_change().shift(-1)[long_positions].fillna(0) * position_size
    short_returns = -price_data.pct_change().shift(-1)[short_positions].fillna(0) * position_size
    strategy_returns = long_returns.sum(axis=1) + short_returns.sum(axis=1)
    
    return strategy_returns, long_positions, short_positions, sentiment_score

# Fetch stock data
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META']
end_date = datetime.now()
start_date = end_date - timedelta(days=365)
price_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Simulate sentiment data
sentiment_data = simulate_sentiment(tickers, len(price_data))

# Calculate volatility
returns = price_data.pct_change()
volatility_data = calculate_volatility(returns)

# Implement strategy
strategy_returns, long_positions, short_positions, sentiment_score = sentiment_volatility_strategy(sentiment_data, price_data, volatility_data)

# Calculate cumulative returns
cumulative_returns = (1 + strategy_returns).cumprod()

# Create interactive plot
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Strategy Cumulative Returns', 'Sentiment Score', 'Long/Short Positions'))

# Plot cumulative returns
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, name='Strategy Returns'), row=1, col=1)

# Plot sentiment score
fig.add_trace(go.Scatter(x=sentiment_score.index, y=sentiment_score, name='Sentiment Score'), row=2, col=1)

# Plot long and short positions
for ticker in tickers:
    fig.add_trace(go.Scatter(x=long_positions.index, y=long_positions[ticker] * 1, name=f'{ticker} Long', mode='markers'), row=3, col=1)
    fig.add_trace(go.Scatter(x=short_positions.index, y=short_positions[ticker] * -1, name=f'{ticker} Short', mode='markers'), row=3, col=1)

fig.update_layout(height=900, title_text="Sentiment-Driven Volatility Strategy")
fig.show()

# Print strategy performance
total_return = cumulative_returns.iloc[-1] - 1
annualized_return = (cumulative_returns.iloc[-1] ** (252 / len(cumulative_returns)) - 1) if len(cumulative_returns) > 0 else 0
sharpe_ratio = (strategy_returns.mean() / strategy_returns.std()) * np.sqrt(252) if strategy_returns.std() != 0 else 0

print(f"Total Return: {total_return:.2%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")


[*********************100%***********************]  5 of 5 completed


Total Return: 0.00%
Annualized Return: 0.00%
Sharpe Ratio: 0.00


In [16]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

# Define the start and end dates
start_date = "2015-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# List of selected stocks
selected_stocks = ['AACG', 'AADI', 'AADR', 'AAL', 'AAME', 'ZZZ', 'ZYXI', 'ZYME', 'ZVSA', 'ZVRA', 'ZURA']

# Function to safely download stock data and handle errors
def safe_download(symbol, start, end):
    try:
        data = yf.download(symbol, start=start, end=end)
        return data
    except Exception as e:
        print(f"Unable to import {symbol}: {str(e)}")
        return None

# Download stock data
stock_data = {symbol: safe_download(symbol, start_date, end_date) for symbol in selected_stocks}

# Remove any stocks that failed to download
valid_stocks = [symbol for symbol, data in stock_data.items() if data is not None]
stock_data = {symbol: data for symbol, data in stock_data.items() if data is not None}

# Calculate monthly returns
monthly_returns = {symbol: data['Adj Close'].resample('M').last().pct_change() 
                   for symbol, data in stock_data.items()}

# Combine all returns into a single DataFrame
all_returns = pd.DataFrame({symbol: returns for symbol, returns in monthly_returns.items()})

# Implement the backtesting strategy
n_long = min(5, len(valid_stocks))
n_short = min(10, len(valid_stocks) - n_long)
long_weights = [1/n_long] * n_long
short_weights = [-1/n_short] * n_short
weights = long_weights + short_weights

# Calculate portfolio returns
portfolio_returns = all_returns.iloc[:, :len(weights)].mul(weights).sum(axis=1)

# Calculate cumulative returns
cumulative_returns = (1 + portfolio_returns).cumprod()

# Calculate annualized return
annualized_return = (cumulative_returns.iloc[-1] ** (1 / (len(cumulative_returns) / 12))) - 1

# Calculate Sharpe ratio (assuming risk-free rate of 0)
sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(12)

# Create an interactive plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=cumulative_returns.index,
    y=cumulative_returns.values,
    mode='lines',
    name='Cumulative Returns'
))

fig.update_layout(
    title='Cumulative Returns of Alphabetical Strategy',
    xaxis_title='Date',
    yaxis_title='Cumulative Returns',
    hovermode='x unified'
)

# Add annotations for key metrics
fig.add_annotation(
    x=1, y=1, xref='paper', yref='paper',
    text=f'Annualized Return: {annualized_return:.2%}<br>Sharpe Ratio: {sharpe_ratio:.2f}',
    showarrow=False,
    font=dict(size=12),
    align='right',
    xanchor='right',
    yanchor='top'
)

# Show the interactive plot
fig.show()

# Print the results
print(f"Cumulative Returns: {cumulative_returns.iloc[-1]:.2f}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Cumulative Returns: 0.79
Annualized Return: -2.40%
Sharpe Ratio: 0.14
